<a href="https://colab.research.google.com/github/hmanoj26/7316/blob/main/Supervised_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap
!pip install ipywidgets
!pip install whiteboxgui

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 6.0 MB/s eta 0:00:00


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# !pip install geemap
# !pip install ipywidgets

import os
import ee
import geemap
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import json
import whiteboxgui


import ipywidgets as widgets
from IPython.display import display

# # Trigger the authentication flow.
ee.Authenticate()

# # Initialize the library.
ee.Initialize(project='remote-sensing-486415')

In [9]:
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [22]:
dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}
map.addLayer(dem, vis_params, 'SRTM DEM', True, 0.5)

## Selecting Landsat Images

**Surface Reflectance** [Landsat Data](https://developers.google.com/earth-engine/datasets/catalog/landsat)

Landsat 9: LANDSAT/LC09/C01/T1_SR

Landsat 8: LANDSAT/LC08/C01/T1_SR

Landsat 7: LANDSAT/LE07/C01/T1_SR

Landsat 5: LANDSAT/LT05/C01/T1_SR


In [23]:
def get_map_feature():
  drawn_features = map.draw_features
  if drawn_features:
      print(f"Found {len(drawn_features)} drawn features.")
      for i, feature in enumerate(drawn_features):
          # print(f"\nFeature {i+1}:")
          # Access the geometry of the drawn feature using .geometry()
          # geometry = feature.geometry()
          # print(json.dumps(geometry.getInfo(), indent=2))

          # You can convert this to an ee.Geometry object if needed
          return ee.Geometry(feature.geometry())
  else:
      print("No features have been drawn on the map yet. Please use the drawing tools on the map to draw a polygon or other shape.")

In [25]:
search_bounds = get_map_feature()

No features have been drawn on the map yet. Please use the drawing tools on the map to draw a polygon or other shape.


Add data to the map

In [32]:
point = ee.Geometry.Point([-122.4439, 37.7538])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(point)
    .filterDate("2016-01-01", "2016-12-31")
    .sort("CLOUD_COVER")
    .first()
    .select("SR_B[1-7]")
)

vis_params = {"min": 0, "max": 3000, "bands": ["SR_B5", "SR_B4", "SR_B3"]}

map.centerObject(point, 8)
map.addLayer(image, vis_params, "Landsat-8")

Check image properties


In [33]:
ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()


'2016-11-18'

In [34]:
image.get("CLOUD_COVER").getInfo()


0.08

Make training dataset
There are several ways you can create a region for generating the training dataset.

Draw a shape (e.g., rectangle) on the map and the use region = Map.user_roi
Define a geometry, such as region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
Create a buffer zone around a point, such as region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)
If you don't define a region, it will use the image footprint by default

In [36]:
region = map.user_roi
region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

In [37]:

nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(image.geometry())
map.addLayer(nlcd, {}, "NLCD")
map

Map(bottom=25636.0, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['positio…

In [38]:
# Make the training dataset.
points = nlcd.sample(
    **{
        "region": image.geometry(),
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

map.addLayer(points, {}, "training", False)

In [39]:
print(points.size().getInfo())


3607


In [40]:
print(points.first().getInfo())


{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-121.75363086398976, 37.774777213706344]}, 'id': '0', 'properties': {'landcover': 71}}


## Train the classifier


In [45]:
# Use these bands for prediction.
bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]


# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

In [46]:
type(training)

ee.featurecollection.FeatureCollection

In [50]:
# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands) #smileCart - A decision tree

In [52]:
type(trained)

ee.classifier.Classifier

In [51]:
print(training.first().getInfo())


{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'SR_B1': 8010, 'SR_B2': 8218, 'SR_B3': 9872, 'SR_B4': 8660, 'SR_B5': 24065, 'SR_B6': 15464, 'SR_B7': 11414, 'landcover': 71}}


## Classify the image

In [49]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
map.addLayer(result.randomVisualizer(), {}, "classified")
map

Map(bottom=25636.0, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['positio…

## Render categorical map
To render a categorical map, we can set two image properties: landcover_class_values and landcover_class_palette. We can use the same style as the NLCD so that it is easy to compare the tw

In [54]:
class_values = nlcd.get("landcover_class_values").getInfo()
class_values

[11,
 12,
 21,
 22,
 23,
 24,
 31,
 41,
 42,
 43,
 51,
 52,
 71,
 72,
 73,
 74,
 81,
 82,
 90,
 95]

In [55]:
class_palette = nlcd.get("landcover_class_palette").getInfo()
class_palette

['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94',
 'dcd93d',
 'ab7028',
 'bad9eb',
 '70a3ba']

In [56]:
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [57]:
map.addLayer(landcover, {}, "Land cover")
map

Map(bottom=51070.0, center=[37.54022177661216, -122.11578369140626], controls=(WidgetControl(options=['positio…

Visualize the result

In [58]:
print("Change layer opacity:")
cluster_layer = map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

## Add a legend to the map

In [59]:
map.add_legend(builtin_legend="NLCD")
map

Map(bottom=51251.0, center=[37.14499280340638, -121.46484375000001], controls=(WidgetControl(options=['positio…

# Export the result

### Export the result directly to your computer:

In [ ]:
import os

out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
out_file = os.path.join(out_dir, "landcover.tif")

In [ ]:
geemap.ee_export_image(landcover, filename=out_file, scale=900)

### Export the result to Google Drive:



In [ ]:
geemap.ee_export_image_to_drive(
    landcover, description="landcover", folder="export", scale=900
)